In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# from io import StringIO # python3; python2: BytesIO 
import boto3
import s3fs
import awscli
import timeit
from datetime import datetime

In [2]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

team3-final-bucket
wshih-bucket


In [3]:
my_bucket = s3.Bucket('team3-final-bucket')

In [4]:
client = boto3.client('s3')
path = 's3://team3-final-bucket/Resources_raw/'

In [5]:
objects_df = pd.read_csv(path + 'objects.csv', dtype={'status':str, 'entity_id':str})
objects_df.head()

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
0,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00
1,c:10,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,NaN,...,NaN,0,0.0,NaN,NaN,0,6,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14
2,c:100,Company,100,NaN,There,there,/company/there,games_video,acquired,NaN,...,NaN,0,0.0,2003-02-01,2011-09-23,4,12,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48
3,c:10000,Company,10000,NaN,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,2008-07-26,...,NaN,0,0.0,NaN,NaN,0,0,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18
4,c:10001,Company,10001,NaN,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,2008-07-26,...,NaN,0,0.0,NaN,NaN,0,0,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18


In [6]:
objects_df.dtypes

id                      object
entity_type             object
entity_id               object
parent_id               object
name                    object
normalized_name         object
permalink               object
category_code           object
status                  object
founded_at              object
closed_at               object
domain                  object
homepage_url            object
twitter_username        object
logo_url                object
logo_width               int64
logo_height              int64
short_description       object
description             object
overview                object
tag_list                object
country_code            object
state_code              object
city                    object
region                  object
first_investment_at     object
last_investment_at      object
investment_rounds        int64
invested_companies       int64
first_funding_at        object
last_funding_at         object
funding_rounds           int64
funding_

In [7]:
objects_df['category_code'].unique()

array(['web', 'games_video', 'network_hosting', 'advertising',
       'cleantech', nan, 'enterprise', 'other', 'consulting', 'mobile',
       'health', 'software', 'analytics', 'finance', 'education',
       'medical', 'manufacturing', 'biotech', 'ecommerce',
       'public_relations', 'hardware', 'search', 'news', 'government',
       'security', 'photo_video', 'travel', 'semiconductor', 'social',
       'legal', 'transportation', 'hospitality', 'sports', 'nonprofit',
       'fashion', 'messaging', 'music', 'automotive', 'design',
       'real_estate', 'local', 'nanotech', 'pets'], dtype=object)

In [8]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [9]:
is_string_dtype(objects_df['status'])

True

In [10]:
is_numeric_dtype(objects_df['status'])

False

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(objects_df.dtypes)

id                      object
entity_type             object
entity_id               object
parent_id               object
name                    object
normalized_name         object
permalink               object
category_code           object
status                  object
founded_at              object
closed_at               object
domain                  object
homepage_url            object
twitter_username        object
logo_url                object
logo_width               int64
logo_height              int64
short_description       object
description             object
overview                object
tag_list                object
country_code            object
state_code              object
city                    object
region                  object
first_investment_at     object
last_investment_at      object
investment_rounds        int64
invested_companies       int64
first_funding_at        object
last_funding_at         object
funding_rounds           int64
funding_

In [12]:
objects_df.head()

#id c:XXXX = company,  r:XXXX=product, f:XXXX=financial org, p:XXXX=person

,id,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
0,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00
1,c:10,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,NaN,...,NaN,0,0.0,NaN,NaN,0,6,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14
2,c:100,Company,100,NaN,There,there,/company/there,games_video,acquired,NaN,...,NaN,0,0.0,2003-02-01,2011-09-23,4,12,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48
3,c:10000,Company,10000,NaN,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,2008-07-26,...,NaN,0,0.0,NaN,NaN,0,0,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18
4,c:10001,Company,10001,NaN,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,2008-07-26,...,NaN,0,0.0,NaN,NaN,0,0,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18


##### Transforming Data

In [13]:
objects_df = objects_df.rename(columns = {'id':'object_id'})
objects_df.head()

,object_id,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
0,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00
1,c:10,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,NaN,...,NaN,0,0.0,NaN,NaN,0,6,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14
2,c:100,Company,100,NaN,There,there,/company/there,games_video,acquired,NaN,...,NaN,0,0.0,2003-02-01,2011-09-23,4,12,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48
3,c:10000,Company,10000,NaN,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,2008-07-26,...,NaN,0,0.0,NaN,NaN,0,0,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18
4,c:10001,Company,10001,NaN,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,2008-07-26,...,NaN,0,0.0,NaN,NaN,0,0,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18


##### Dropping redundant columns

In [14]:
objects_clean = objects_df.drop(columns = {'permalink', 'created_by', 'created_at', 'updated_at', 'logo_url', 'logo_width', 'logo_height', 'overview', 'tag_list', 'domain', 'twitter_username', 'homepage_url', 'normalized_name', 'short_description','description'})
objects_clean

,object_id,entity_type,entity_id,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,...,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,9995,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,NaN,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462647,r:9996,Product,9996,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,NaN,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462648,r:9997,Product,9997,c:14164,"Cmail, broadcast email marketing",NaN,operating,NaN,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462649,r:9998,Product,9998,c:14164,"CatylistCRM, contact database",NaN,operating,NaN,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


##### Converting date columns into date format

In [15]:
# Converting strings to date type
objects_clean['founded_at'] = pd.to_datetime(objects_clean.founded_at, format='%Y-%m-%d')
objects_clean['closed_at'] = pd.to_datetime(objects_clean.closed_at, format='%Y-%m-%d')
objects_clean['first_investment_at'] = pd.to_datetime(objects_clean.first_investment_at, format='%Y-%m-%d')
objects_clean['last_investment_at'] = pd.to_datetime(objects_clean.last_investment_at, format='%Y-%m-%d')
objects_clean['first_funding_at'] = pd.to_datetime(objects_clean.first_funding_at, format='%Y-%m-%d')
objects_clean['last_funding_at'] = pd.to_datetime(objects_clean.last_funding_at, format='%Y-%m-%d')
objects_clean['first_milestone_at'] = pd.to_datetime(objects_clean.first_milestone_at, format='%Y-%m-%d')
objects_clean['last_milestone_at'] = pd.to_datetime(objects_clean.last_milestone_at, format='%Y-%m-%d')

In [16]:
objects_clean.dtypes

object_id                      object
entity_type                    object
entity_id                      object
parent_id                      object
name                           object
category_code                  object
status                         object
founded_at             datetime64[ns]
closed_at              datetime64[ns]
country_code                   object
state_code                     object
city                           object
region                         object
first_investment_at    datetime64[ns]
last_investment_at     datetime64[ns]
investment_rounds               int64
invested_companies              int64
first_funding_at       datetime64[ns]
last_funding_at        datetime64[ns]
funding_rounds                  int64
funding_total_usd             float64
first_milestone_at     datetime64[ns]
last_milestone_at      datetime64[ns]
milestones                      int64
relationships                   int64
dtype: object

In [17]:
# Changing columns order
objects_clean.columns.to_list()

['object_id',
 'entity_type',
 'entity_id',
 'parent_id',
 'name',
 'category_code',
 'status',
 'founded_at',
 'closed_at',
 'country_code',
 'state_code',
 'city',
 'region',
 'first_investment_at',
 'last_investment_at',
 'investment_rounds',
 'invested_companies',
 'first_funding_at',
 'last_funding_at',
 'funding_rounds',
 'funding_total_usd',
 'first_milestone_at',
 'last_milestone_at',
 'milestones',
 'relationships']

In [18]:
# Adding a new column for collecting dates

objects_clean["end_dates"] = ""
objects_clean.head()

,object_id,entity_type,entity_id,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,end_dates
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-17,NaT,USA,...,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17,
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaT,NaT,USA,...,0,NaT,NaT,0,0.0,NaT,NaT,0,6,
2,c:100,Company,100,NaN,There,games_video,acquired,NaT,NaT,USA,...,0,NaT,NaT,0,0.0,2003-02-01,2011-09-23,4,12,
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,NaN,...,0,NaT,NaT,0,0.0,NaT,NaT,0,0,
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,NaN,...,0,NaT,NaT,0,0.0,NaT,NaT,0,0,


In [19]:
objects_clean2 = objects_clean[['object_id',
 'entity_type',
 'entity_id',
 'parent_id',
 'name',
 'category_code',
 'status',
 'founded_at',
 'first_investment_at',
 'first_funding_at',
 'country_code',
 'state_code',
 'city',
 'region',
 'investment_rounds',
 'invested_companies',
 'funding_rounds',
 'end_dates',                               
 'closed_at',
 'last_investment_at',
 'last_funding_at',
 'funding_total_usd',
 'first_milestone_at',
 'last_milestone_at',
 'milestones',
 'relationships']]

In [20]:
objects_clean2.head()

,object_id,entity_type,entity_id,parent_id,name,category_code,status,founded_at,first_investment_at,first_funding_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-17,NaT,2005-10-01,...,3,,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaT,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,NaT,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0


In [21]:
#objects_clean2.dtypes

In [22]:
# Filtering 'companies' only
companies_df = objects_clean2[objects_clean2['entity_type']=='Company']
companies_df

,object_id,entity_type,entity_id,parent_id,name,category_code,status,founded_at,first_investment_at,first_funding_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-17,NaT,2005-10-01,...,3,,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaT,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,NaT,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,99940,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,9995,NaN,FohBoh,public_relations,operating,2007-11-01,NaT,2008-08-25,...,1,,NaT,NaT,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,9996,NaN,CSC,consulting,operating,1959-01-01,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,9997,NaN,Top-candidate,search,operating,2008-07-01,NaT,NaT,...,0,,NaT,NaT,NaT,0.0,NaT,NaT,0,1


##### Working on filling in gap of end dates = pulling from different columns and related tables

In [34]:
# Filling out end dates gaps by combining dates from three columns of 'Companies':
# closed_at, last_investment_at, last_funding_at.
# If conflicting dates are found in one record, the latest is taken.
def maxdate(a,b,c):
    if a == 'NaT':
        return max(b,c)
    elif b == 'NaT':
        return max(a,c)
    else:
        return c


#def maxdate(a,b,c):
 #   return max(a,b,c)

In [35]:
# function returns 'NaT' as max value. Therefore have to give arbitrary max date to compare to

companies_df['end_dates'] = companies_df.apply(lambda row : maxdate(row['closed_at'], 
                      row['last_investment_at'], row['last_funding_at']), axis = 1)

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [38]:
companies_df

,object_id,entity_type,entity_id,parent_id,name,category_code,status,founded_at,first_investment_at,first_funding_at,...,funding_rounds,end_dates,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,1,NaN,Wetpaint,web,operating,2005-10-17,NaT,2005-10-01,...,3,2008-05-19,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,10,NaN,Flektor,games_video,acquired,NaT,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,100,NaN,There,games_video,acquired,NaT,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,10000,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,10001,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,99940,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,9995,NaN,FohBoh,public_relations,operating,2007-11-01,NaT,2008-08-25,...,1,2008-08-25,NaT,NaT,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,9996,NaN,CSC,consulting,operating,1959-01-01,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,9997,NaN,Top-candidate,search,operating,2008-07-01,NaT,NaT,...,0,NaT,NaT,NaT,NaT,0.0,NaT,NaT,0,1
